# Data Ingestion

In [2]:
!pip install langchain_core

^C


  Using cached langsmith-0.4.41-py3-none-any.whl.metadata (14 kB)
  Using cached pydantic-2.12.4-py3-none-any.whl.metadata (89 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 441.3 kB/s eta 0:00:04
   ---------- ----------------------------- 0.5/2.0 MB 441.3 kB/s eta 0:00:04
   --------------- ------------------------ 0.8/2.0 MB 459.5 kB/s eta 0:00:03
   --------------- ------------------------ 0.8/2.0 MB 459.5 kB/s eta 0:00:03

In [4]:
## document structure


from langchain_core.documents import Document 

In [8]:
doc=Document(
    page_content="This is the content of the document.",
    metadata={"source": "generated", "author": "AI"},
)

In [9]:
doc

Document(metadata={'source': 'generated', 'author': 'AI'}, page_content='This is the content of the document.')

In [11]:
## creating a simple txt document

import os
os.makedirs("../data/text_files", exist_ok=True)

In [ ]:
sample_text = {
    
    "../data/text_files/doc1.txt": """This is the content of document , it is about python programming 
and its applications in data science.
Python is a versatile language that is widely used in various fields.
keyfeatures include simplicity, readability, and a vast ecosystem of libraries.
  
it has verstaile uses including web development, data analysis, machine learning, and automation.
  
    """,

}


for file_path, content in sample_text.items():   # using 2 variables, one for key and one for value
    with open(file_path, "w",encoding="utf-8") as f:
        f.write(content)
        
print("Sample text files created.")


Document(metadata={'source': 'generated', 'author': 'AI'}, page_content='This is the content of the document.')